#### Imports

In [1]:
from random import randint
import pandas as pd
import sys

from alice import Alice
from bob import Bob

from partial_key_recovery import attack

sys.path.append('..')

from metrics import key_recovery_metrics
from qkd import execute_qkd

#### Execute the LL20 QKD protocol

In [33]:
public, private = execute_qkd(Alice, Bob, 16, DEBUG = True)

Key exchange completed, here's the data:
Alice send pairs: {0: '1x,0z', 1: '1x,0z', 2: '0x,0z', 3: '1x,0z', 4: '1x,1z', 5: '0x,0z', 6: '0x,0z', 7: '0x,0z', 8: '1x,1z', 9: '0x,1z', 10: '1x,0z', 11: '1x,0z', 12: '1x,0z', 13: '0x,1z', 14: '0x,0z', 15: '1x,0z'}
Bob send double matchings: [3, 7, 11, 13, 15]
Alice send usable frames: [(3, 13), (11, 13), (13, 15)]
Bob send sifting string: ['00', '10', '00']
Bob send measured string: ['00', '10', '00']
Bob shared key: 110010
Alice shared key: 110010


#### Execute the LL20 QKD protocol and the Partial Key Recovery attack

* Error-free

In [12]:
# Assuming error-free

VALID_SS = [
    "00,00",
    "00,11",
    "01,10",
    "01,01",
    "10,01",
    "10,10",
    "11,11"
]

key_recovery_metrics(execute_qkd, Alice, Bob, 16, attack, 2, VALID_SS)

{'double_matchings': [0, 4, 5, 6, 7, 8, 11],
 'bits_recovered': 4,
 'bits': 22,
 '%': 0.18181818181818182,
 'SS_left': ['01,10',
  '01,10',
  '01,10',
  '10,10',
  '00,00',
  '10,01',
  '00,00',
  '10,01',
  '10,10']}

* Assuming errors. Then, the Partial Key Recovery becomes a total Key Recovery

In [13]:
VALID_SS = [
    "00,11",
    "11,11"
]

key_recovery_metrics(execute_qkd, Alice, Bob, 64, attack, 2, VALID_SS, depolarize_probability = 1)

Key exchange failed, both keys are not equal. Please, try again.


{'double_matchings': [1,
  3,
  4,
  5,
  6,
  8,
  13,
  15,
  16,
  19,
  21,
  22,
  23,
  24,
  28,
  30,
  31,
  32,
  33,
  36,
  37,
  39,
  40,
  45,
  46,
  47,
  50,
  51,
  55,
  57,
  62],
 'bits_recovered': 108,
 'bits': 108,
 '%': 1.0,
 'SS_left': []}

#### Add results to dataframe

* Error-free

In [2]:
VALID_SS = [
    "00,00",
    "00,11",
    "01,10",
    "01,01",
    "10,01",
    "10,10",
    "11,11"
]

try:
    df = pd.read_csv("results_error_free.csv")
except:
    file = open("results_error_free.csv", "w")
    file.write("double_matchings,bits_recovered,bits,%")
    file.close()
    df = pd.read_csv("results_error_free.csv")

while True:
    pairs = randint(8, 256)
    results = key_recovery_metrics(execute_qkd, Alice, Bob, pairs, attack, 2, VALID_SS)
    
    if not results: continue

    df.loc[len(df)] = [ len(results["double_matchings"]), results["bits_recovered"], results["bits"], results["%"] ]
    
    df.to_csv("results_error_free.csv", index = False)

Key exchange failed, there's no usable frames. Please, try again.
Key exchange failed, there's no usable frames. Please, try again.
Key exchange failed, there's no usable frames. Please, try again.
Key exchange failed, there's no usable frames. Please, try again.


KeyboardInterrupt: 

* Assuming errors in the quantum channel

In [3]:
VALID_SS = [
    "00,11",
    "11,11"
]

try:
    df = pd.read_csv("results_error.csv")
except:
    file = open("results_error.csv", "w")
    file.write("double_matchings,bits_recovered,bits,%")
    file.close()
    df = pd.read_csv("results_error.csv")

while True:
    pairs = randint(8, 256)
    results = key_recovery_metrics(execute_qkd, Alice, Bob, pairs, attack, 2, VALID_SS, depolarize_probability = 1)
    
    if not results: continue

    df.loc[len(df)] = [ len(results["double_matchings"]), results["bits_recovered"], results["bits"], results["%"] ]
    
    df.to_csv("results_error.csv", index = False)

Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f82a5c27250>>
Traceback (most recent call last):
  File "/home/kali/anaconda3/envs/sage/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, there's no usable frames. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, bot

KeyboardInterrupt: 